In [ ]:
import warnings
warnings.filterwarnings('ignore')

XGB

In [ ]:
import pandas as pd

# 파일 경로
train_path = "train.csv"
test_path = "test.csv"

# 데이터 로드
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# 데이터 구조 확인
train_info = train_df.info()
test_info = test_df.info()

# train 데이터 상위 5개 확인
train_head = train_df.head()

train_info, test_info, train_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                10000 non-null  object 
 1   주거 형태              10000 non-null  object 
 2   연간 소득              10000 non-null  float64
 3   현재 직장 근속 연수        10000 non-null  object 
 4   체납 세금 압류 횟수        10000 non-null  float64
 5   개설된 신용계좌 수         10000 non-null  int64  
 6   신용 거래 연수           10000 non-null  float64
 7   최대 신용한도            10000 non-null  float64
 8   신용 문제 발생 횟수        10000 non-null  int64  
 9   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 10  개인 파산 횟수           10000 non-null  int64  
 11  대출 목적              10000 non-null  object 
 12  대출 상환 기간           10000 non-null  object 
 13  현재 대출 잔액           10000 non-null  float64
 14  현재 미상환 신용액         10000 non-null  float64
 15  월 상환 부채액           10000 non-null  float64
 16  신용 점수              1000

(None,
 None,
            UID 주거 형태      연간 소득 현재 직장 근속 연수  체납 세금 압류 횟수  개설된 신용계좌 수  \
 0  TRAIN_00000    자가  1941337.5      10년 이상          0.0           9   
 1  TRAIN_00001    월세  1979505.0      10년 이상          0.0           5   
 2  TRAIN_00002    월세  1356381.0          4년          0.0          12   
 3  TRAIN_00003    월세  1049017.5          6년          0.0          15   
 4  TRAIN_00004    월세  4320217.5          2년          0.0          11   
 
    신용 거래 연수   최대 신용한도  신용 문제 발생 횟수  마지막 연체 이후 경과 개월 수  개인 파산 횟수  대출 목적  \
 0      13.4  400597.5            0                 24         1  부채 통합   
 1      15.1  360679.5            0                 11         0  부채 통합   
 2      18.8  491770.5            1                 74         3  부채 통합   
 3      14.8  411546.0            1                 22         1  부채 통합   
 4      26.1  895288.5            0                 32         0  부채 통합   
 
   대출 상환 기간   현재 대출 잔액  현재 미상환 신용액  월 상환 부채액  신용 점수  채무 불이행 여부  
 0    단기 상환   390903.0    225

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. 불필요한 컬럼 제거
train_df = train_df.drop(columns=["UID"])
test_UID=test_df.pop('UID')

# 2. 범주형 변수 처리 (레이블 인코딩)
categorical_cols = ["주거 형태", "현재 직장 근속 연수", "대출 목적", "대출 상환 기간"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

# 3. 특징(X)과 타겟(y) 분리
X = train_df.drop(columns=["채무 불이행 여부"])
y = train_df["채무 불이행 여부"]
X_test = test_df.copy()

# 4. 데이터 분할 (학습용/검증용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 5. 스케일링 (표준화)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# 데이터 전처리 완료
X_train_scaled.shape, X_valid_scaled.shape, X_test_scaled.shape

((8000, 16), (2000, 16), (2062, 16))

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# XGBoost 모델 학습
model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss")
model.fit(X_train_scaled, y_train)

# 검증 데이터에 대한 예측
y_valid_pred = model.predict_proba(X_valid_scaled)[:, 1]

# ROC-AUC 평가
roc_auc = roc_auc_score(y_valid, y_valid_pred)
roc_auc

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:18:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


0.7058504176326879

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# XGBoost 하이퍼파라미터 튜닝을 위한 파라미터 그리드
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9]
}

# RandomizedSearchCV 실행
xgb = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)
random_search = RandomizedSearchCV(xgb, param_grid, n_iter=10, scoring="roc_auc", cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train_scaled, y_train)

# 최적 파라미터 및 성능 확인
best_params = random_search.best_params_
best_score = random_search.best_score_

best_params, best_score

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:19:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


({'subsample': 0.9,
  'n_estimators': 200,
  'max_depth': 3,
  'learning_rate': 0.05,
  'colsample_bytree': 0.8},
 0.7427826617478358)

In [ ]:
# 최적 하이퍼파라미터로 XGBoost 모델 재학습
best_model = XGBClassifier(**best_params, random_state=42, eval_metric="logloss")
best_model.fit(X_train_scaled, y_train)

# 검증 데이터 평가
y_valid_pred = best_model.predict_proba(X_valid_scaled)[:, 1]
roc_auc_final = roc_auc_score(y_valid, y_valid_pred)

# 테스트 데이터 예측
y_test_pred = best_model.predict_proba(X_test_scaled)[:, 1]

roc_auc_final

0.750733137829912

In [ ]:
import pandas as pd

# 예측 결과 저장
result10 = pd.DataFrame({"UID": test_UID, "채무 불이행 확률": y_test_pred})
result10.to_csv("result10.csv", index=False)

XGB2

In [ ]:
import pandas as pd

# 데이터 다시 로드
train_path = "train.csv"
test_path = "test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# 데이터 확인
train_df.head(), test_df.head()

(           UID 주거 형태      연간 소득 현재 직장 근속 연수  체납 세금 압류 횟수  개설된 신용계좌 수  \
 0  TRAIN_00000    자가  1941337.5      10년 이상          0.0           9   
 1  TRAIN_00001    월세  1979505.0      10년 이상          0.0           5   
 2  TRAIN_00002    월세  1356381.0          4년          0.0          12   
 3  TRAIN_00003    월세  1049017.5          6년          0.0          15   
 4  TRAIN_00004    월세  4320217.5          2년          0.0          11   
 
    신용 거래 연수   최대 신용한도  신용 문제 발생 횟수  마지막 연체 이후 경과 개월 수  개인 파산 횟수  대출 목적  \
 0      13.4  400597.5            0                 24         1  부채 통합   
 1      15.1  360679.5            0                 11         0  부채 통합   
 2      18.8  491770.5            1                 74         3  부채 통합   
 3      14.8  411546.0            1                 22         1  부채 통합   
 4      26.1  895288.5            0                 32         0  부채 통합   
 
   대출 상환 기간   현재 대출 잔액  현재 미상환 신용액  월 상환 부채액  신용 점수  채무 불이행 여부  
 0    단기 상환   390903.0    225457.5    8806.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# 목표 변수 분리
X = train_df.drop(columns=["UID", "채무 불이행 여부"])
y = train_df["채무 불이행 여부"]

# 테스트 데이터 준비 (UID 제외)
test_UID=test_df.pop('UID')
X_test=test_df

# 범주형 변수 인코딩
categorical_cols = ["주거 형태", "현재 직장 근속 연수", "대출 목적", "대출 상환 기간"]
encoder = OneHotEncoder(sparse_output=False, drop="first")

# 먼저 fit()을 실행하여 encoder를 학습
encoder.fit(X[categorical_cols])

# 변환 적용
X_encoded = encoder.transform(X[categorical_cols])
X_test_encoded = encoder.transform(X_test[categorical_cols])

# 인코딩된 컬럼명을 가져오기
encoded_cols = encoder.get_feature_names_out(categorical_cols)

# DataFrame으로 변환
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_cols, index=X.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoded_cols, index=X_test.index)

# 기존 데이터에서 범주형 변수 제거 후 결합
X = X.drop(columns=categorical_cols).reset_index(drop=True)
X_test = X_test.drop(columns=categorical_cols).reset_index(drop=True)

X = pd.concat([X, X_encoded_df], axis=1)
X_test = pd.concat([X_test, X_test_encoded_df], axis=1)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# 훈련 데이터 분할 (Train / Validation)
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 데이터 전처리 완료
X_train.shape, X_valid.shape, X_test_scaled.shape

((8000, 39), (2000, 39), (2062, 39))

In [ ]:
from xgboost import XGBClassifier
import numpy as np

# XGBoost 모델 학습 (기본 설정)
model = XGBClassifier(random_state=42, eval_metric="logloss")
model.fit(X_train, y_train)

# 특성 중요도 가져오기
feature_importances = model.feature_importances_
feature_names = X.columns

# 중요도 높은 순서로 정렬
important_features = pd.DataFrame({"Feature": feature_names, "Importance": feature_importances})
important_features = important_features.sort_values(by="Importance", ascending=False)

# 상위 10개 특성 출력
important_features.head(10)

,Feature,Importance
28,대출 목적_부채 통합,0.152295
38,대출 상환 기간_장기 상환,0.055755
15,현재 직장 근속 연수_1년,0.041954
17,현재 직장 근속 연수_2년,0.041669
5,신용 문제 발생 횟수,0.038840
21,현재 직장 근속 연수_6년,0.038352
2,개설된 신용계좌 수,0.037350
7,개인 파산 횟수,0.032672
0,연간 소득,0.031364
18,현재 직장 근속 연수_3년,0.027866


In [ ]:
# 중요도 0.02 이하 제거
low_importance_features = important_features[important_features["Importance"] < 0.02]["Feature"].tolist()
X = X.drop(columns=low_importance_features)
X_test = X_test.drop(columns=low_importance_features)

# 데이터 재스케일링
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# 훈련 데이터 재분할
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# XGBoost 하이퍼파라미터 튜닝
param_grid = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9]
}

xgb = XGBClassifier(random_state=42, eval_metric="logloss")
random_search = RandomizedSearchCV(xgb, param_grid, n_iter=10, scoring="roc_auc", cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# 최적 파라미터로 모델 재학습
best_params = random_search.best_params_
best_model = XGBClassifier(**best_params, random_state=42, eval_metric="logloss")
best_model.fit(X_train, y_train)

# 검증 데이터 평가
y_valid_pred = best_model.predict_proba(X_valid)[:, 1]
roc_auc_final = roc_auc_score(y_valid, y_valid_pred)

# 테스트 데이터 예측
y_test_pred = best_model.predict_proba(X_test_scaled)[:, 1]

# 최종 ROC-AUC 값 확인
roc_auc_final

0.7433933771977133

In [ ]:
import pandas as pd
import numpy as np
import shap

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import dask
import lightgbm as lgb
import catboost as cb

# 데이터 로드
train_path = "train.csv"
test_path = "test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# 목표 변수 분리
X = train_df.drop(columns=["UID", "채무 불이행 여부"])
y = train_df["채무 불이행 여부"]

# 테스트 데이터 준비 (UID 제외)
test_UID = test_df.pop("UID")
X_test = test_df

# 범주형 변수 인코딩
categorical_cols = ["주거 형태", "현재 직장 근속 연수", "대출 목적", "대출 상환 기간"]
encoder = OneHotEncoder(sparse_output=False, drop="first")

# 변환 적용
X_encoded = encoder.fit_transform(X[categorical_cols])
X_test_encoded = encoder.transform(X_test[categorical_cols])

# 인코딩된 컬럼명을 가져오기
encoded_cols = encoder.get_feature_names_out(categorical_cols)

# DataFrame으로 변환
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_cols, index=X.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoded_cols, index=X_test.index)

# 기존 데이터에서 범주형 변수 제거 후 결합
X = X.drop(columns=categorical_cols).reset_index(drop=True)
X_test = X_test.drop(columns=categorical_cols).reset_index(drop=True)

X = pd.concat([X, X_encoded_df], axis=1)
X_test = pd.concat([X_test, X_test_encoded_df], axis=1)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# 훈련 데이터 분할 (Train / Validation)
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# XGBoost 모델 학습 (기본 설정)
xgb_model = XGBClassifier(random_state=42, eval_metric="logloss")
xgb_model.fit(X_train, y_train)

# SHAP 분석을 위한 객체 생성
explainer = shap.Explainer(xgb_model, X_train)
shap_values = explainer(X_train)

# 평균 절대 SHAP 값 계산 (특성 중요도)
shap_importance = np.abs(shap_values.values).mean(axis=0)

# SHAP 값이 작은 (중요도가 낮은) 특성 제거
threshold = np.percentile(shap_importance, 25)  # 하위 25% 특성 제거
low_shap_features = X.columns[shap_importance < threshold]

# 중요도가 낮은 특성 제거
X = X.drop(columns=low_shap_features)
X_test = X_test.drop(columns=low_shap_features)

# 데이터 재스케일링
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# 훈련 데이터 재분할
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# XGBoost, LightGBM, CatBoost 비교
models = {
    "XGBoost": XGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.05, subsample=0.8,
                             colsample_bytree=0.8, min_child_weight=3, gamma=0.1, scale_pos_weight=1.2,
                             random_state=42, eval_metric="logloss"),
    "LightGBM": lgb.LGBMClassifier(n_estimators=300, max_depth=5, learning_rate=0.05, subsample=0.8,
                                   colsample_bytree=0.8, min_child_weight=3, reg_alpha=0.1,
                                   random_state=42),
    "CatBoost": cb.CatBoostClassifier(n_estimators=300, depth=5, learning_rate=0.05, subsample=0.8,
                                      colsample_bylevel=0.8, l2_leaf_reg=3, random_seed=42, verbose=0)
}

# K-Fold Cross Validation을 사용하여 성능 비교
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring="roc_auc", n_jobs=-1)
    results[name] = np.mean(scores)

results

 99%|===================| 7908/8000 [00:36<00:00]       

{'XGBoost': 0.7370873907875806,
 'LightGBM': 0.7346066962765254,
 'CatBoost': 0.7482797089058948}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# 🎯 XGBoost 최적 파라미터 찾기
xgb_param_grid = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9]
}
xgb = XGBClassifier(random_state=42, eval_metric="logloss")
xgb_search = RandomizedSearchCV(xgb, xgb_param_grid, n_iter=10, scoring="roc_auc", cv=3, random_state=42, n_jobs=-1)
xgb_search.fit(X_train, y_train)
best_xgb_params = xgb_search.best_params_

# 🎯 LightGBM 최적 파라미터 찾기
lgb_param_grid = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9]
}
lgb = LGBMClassifier(random_state=42)
lgb_search = RandomizedSearchCV(lgb, lgb_param_grid, n_iter=10, scoring="roc_auc", cv=3, random_state=42, n_jobs=-1)
lgb_search.fit(X_train, y_train)
best_lgb_params = lgb_search.best_params_

# 🎯 CatBoost 최적 파라미터 찾기
cat_param_grid = {
    "iterations": [100, 300, 500],
    "depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1]
}
cat = CatBoostClassifier(random_state=42, verbose=0)
cat_search = RandomizedSearchCV(cat, cat_param_grid, n_iter=10, scoring="roc_auc", cv=3, random_state=42, n_jobs=-1)
cat_search.fit(X_train, y_train)
best_cat_params = cat_search.best_params_

[LightGBM] [Info] Number of positive: 2730, number of negative: 5270
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1854
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.341250 -> initscore=-0.657729
[LightGBM] [Info] Start training from score -0.657729
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# 각 모델의 최적 파라미터 사용 (이미 튜닝된 best_params 적용)
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, eval_metric="logloss")
lgb_model = LGBMClassifier(**best_lgb_params, random_state=42)
cat_model = CatBoostClassifier(**best_cat_params, random_state=42, verbose=0)

# 모델 학습
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)
cat_model.fit(X_train, y_train)

# 검증 데이터 예측 (확률 값 추출)
y_valid_xgb = xgb_model.predict_proba(X_valid)[:, 1]
y_valid_lgb = lgb_model.predict_proba(X_valid)[:, 1]
y_valid_cat = cat_model.predict_proba(X_valid)[:, 1]

# Soft Voting (가중 평균)
final_valid_pred = (0.3 * y_valid_xgb) + (0.2 * y_valid_lgb) + (0.5 * y_valid_cat)

# 최종 성능 평가
roc_auc_ensemble = roc_auc_score(y_valid, final_valid_pred)
print("앙상블 모델 ROC-AUC:", roc_auc_ensemble)

# 테스트 데이터 예측
y_test_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]
y_test_lgb = lgb_model.predict_proba(X_test_scaled)[:, 1]
y_test_cat = cat_model.predict_proba(X_test_scaled)[:, 1]

# 최종 테스트 데이터 예측 (가중 평균)
y_test_final = (0.3 * y_test_xgb) + (0.2 * y_test_lgb) + (0.5 * y_test_cat)

# 제출 파일 생성
submission = pd.DataFrame({"UID": test_UID, "채무 불이행 확률": y_test_final})
submission.to_csv("submission.csv", index=False)

print("앙상블 모델 예측 완료! 결과가 submission.csv에 저장되었습니다.")

[LightGBM] [Info] Number of positive: 2730, number of negative: 5270
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002748 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1854
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.341250 -> initscore=-0.657729
[LightGBM] [Info] Start training from score -0.657729
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/lightgbm/sklearn.py:1108: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  X = _LGBMValidateData(
/usr/local/lib/python3.11/dist-packages/lightgbm/sklearn.py:1108: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  X = _LGBMValidateData(
